In [1]:
from pathlib import Path
from dateutil.relativedelta import relativedelta
import datetime as dt
import pandas as pd


ROOT_DIR = Path("../../")
DATA_DIR = ROOT_DIR / "data"
INPUT_DATA = DATA_DIR / "00_preraw" / "DAYTON_hourly.csv"
OUTPUT_DATA = DATA_DIR / "01_raw" / "DAYTON_power_consumption.csv"

TRAIN_DATA = DATA_DIR / "02_intermediate" / "train_power.csv"
CV_DATA = DATA_DIR / "02_intermediate" / "cv_power.csv"
TEST_DATA = DATA_DIR / "02_intermediate" / "test_power.csv"

AVG_DAYTON_YEARLY_HOUSEHOLD_KWH = 13092


In [2]:
data = (
    pd.read_csv(INPUT_DATA, parse_dates=["Datetime"])
    .rename(columns={"Datetime": "datetime", "DAYTON_MW": "power_mw"})
)

data.head()

,datetime,power_mw
0,2004-12-31 01:00:00,1596.0
1,2004-12-31 02:00:00,1517.0
2,2004-12-31 03:00:00,1486.0
3,2004-12-31 04:00:00,1469.0
4,2004-12-31 05:00:00,1472.0


In [3]:
min_ts, max_ts = min(data.datetime), max(data.datetime)
min_ts, max_ts

(Timestamp('2004-10-01 01:00:00'), Timestamp('2018-08-03 00:00:00'))

In [4]:
min_date_for_avg = max_ts.date() - relativedelta(years=1)
min_dt_for_avg = dt.datetime.combine(min_date_for_avg, dt.time())
min_dt_for_avg

datetime.datetime(2017, 8, 3, 0, 0)

In [5]:
last_year_data_mwh = (
    data[data.datetime > min_dt_for_avg]
    .power_mw.sum()
)
last_year_data_kwh = last_year_data_mwh * 1000

last_year_data_kwh

17763208000.0

In [6]:
factor = AVG_DAYTON_YEARLY_HOUSEHOLD_KWH / last_year_data_kwh

In [7]:
new_data = (
    data.assign(power_kw=lambda d: d.power_mw * factor)
    .drop(columns=["power_mw"])
)

new_data.head()

,datetime,power_kw
0,2004-12-31 01:00:00,0.001176
1,2004-12-31 02:00:00,0.001118
2,2004-12-31 03:00:00,0.001095
3,2004-12-31 04:00:00,0.001083
4,2004-12-31 05:00:00,0.001085


In [8]:
new_data.to_csv(OUTPUT_DATA)

In [9]:
train_start_dt = dt.datetime(2005, 8, 3, 0, 0)
cv_start_dt = dt.datetime(2015, 8, 3, 0, 0)
test_start_dt = dt.datetime(2017, 8, 3, 0, 0)

train_data = new_data[(new_data.datetime >= train_start_dt) & (new_data.datetime < cv_start_dt)]
cv_data = new_data[(new_data.datetime >= cv_start_dt) & (new_data.datetime < test_start_dt)]
test_data = new_data[new_data.datetime >= test_start_dt]

train_data.to_csv(TRAIN_DATA)
cv_data.to_csv(CV_DATA)
test_data.to_csv(TEST_DATA)